## 🎬 **Demostración:** **Creamos la Base de Datos para Ingresar nuestra DATA**

Para crear la base de datos primero preparamos su entorno, instalamos el DBCODE, el cual nos va a pedir que se intregre a nuestro entorno las librerías que se necesitan, creamos una nueva base de datos la misma que llevará el nombre de **senadores_house_logs**, creamos su tabala con 5 campos para poder ingresar la data en ella y poder analizarla.

In [ ]:
import psycopg2
import json
from datetime import datetime

# Paso 1: Crear la base de datos y la tabla desde Python
def create_database_and_table():
    try:
        # Conexión al servidor PostgreSQL (sin especificar una base de datos)
        conn = psycopg2.connect(
            user="postgres", # Usuario predeterminado
            password="postgres",
            host="localhost",
            port="5432"
        )
        conn.autocommit = True # Necesario para crear una base de datos
        cursor = conn.cursor()

        # Crear la base de datos "SENADORES_HOUSE_logs" si no existe
        cursor.execute("SELECT datname FROM pg_database WHERE datname='senadores_house_logs';")
        if not cursor.fetchone():
            cursor.execute("CREATE DATABASE senadores_house_logs;")
            print("Base de datos 'senadores_house_logs' creada correctamente.")
        else:
            print("La base de datos 'senadores_house_logs' ya existe.")

        # Cerrar la conexión inicial
        cursor.close()
        conn.close()

        # Conectar a la nueva base de datos
        conn = psycopg2.connect(
        database="senadores_house_logs",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
        )
        cursor = conn.cursor()

        cursor.execute("DROP TABLE IF EXISTS structured_logs;")

        # Crear la tabla "structured_logs" si no existe
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS structured_logs (
        id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        committees VARCHAR(250),
        distrit INTEGER,
        party TEXT
        );
        """)
        print("Tabla 'structured_logs' creada correctamente.")

        # Confirmar cambios y cerrar la conexión
        conn.commit()
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error al crear la base de datos o la tabla: {e}")

# Paso 2: Insertar datos estructurados en la tabla
def insert_structured_logs():
    try:
        # Conectar a la base de datos
        conn = psycopg2.connect(
        database="senadores_house_logs",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
        )
        cursor = conn.cursor()        

        # Insertar registros en la tabla de acuerdo a la data extraída
        cursor.executemany("""
            INSERT INTO structured_logs (name, committees, distrit, party)
            VALUES (%s, %s, %s, %s);
        """, members[:total_mimbros])

        # Confirmar cambios y cerrar la conexión
        conn.commit()
        print("Datos estructurados insertados correctamente.")
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error al insertar datos estructurados: {e}")

if __name__ == "__main__":
    create_database_and_table() # Crear base de datos y tabla
    insert_structured_logs() # Insertar datos estructurados
    # save_unstructured_logs() # Guardar logs no estructurados

Como se observa la Data ingresada a nuestra nueva base de Datos es de forma automática esto va a depender de la cantidad de información que tenga la página a ser analizada.

#### De preferencia se debe de verificar que la Data extraída y a ser analizada no sea muy pesada

Echemos un vistazo a lo que tenemos en  `members`. Se corrobora que la Data ingresada es la mimsa del portal en análisis

In [ ]:
print(members[:5])

### **Obtenemos la Data de la BASE para analizarla Gráficamente**
Obtenemos la Data de la nueva base **senadores_house_logs**
Se observa el % de Senadores Demócratas Vs Republicanos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Conectar a PostgreSQL y obtener los datos
def get_data_from_db():
    conn = psycopg2.connect(
        database="senadores_house_logs",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    
    # Obtener distribución de senadores/casa por partido
    cursor.execute("SELECT party, COUNT(*) FROM structured_logs GROUP BY party;")
    party_data = cursor.fetchall()
    
    # Obtener distribución de senadores/casa por distrito
    cursor.execute("SELECT distrit, COUNT(*) FROM structured_logs GROUP BY distrit;")
    district_data = cursor.fetchall()
    
    conn.close()
    return party_data, district_data

# Graficar distribución de senadores/casa por partido
def plot_pie_chart(party_data):
    labels, sizes = zip(*party_data)
    plt.figure(figsize=(5,5))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
    plt.title("Distribución de Senadores/Casa por Partido")
    plt.show()
if __name__ == "__main__":
    party_data, district_data = get_data_from_db()
    plot_pie_chart(party_data)